In [1]:
# importing libraries
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
Файл с данными находится по этому адресу:
    
https://data-in.ru/data-catalog/datasets/201/#

In [2]:
df=pd.read_csv('data.csv', sep=';', index_col=None, decimal=',')

ParserError: Error tokenizing data. C error: Unknown error in IO callback

In [ ]:
df['month'] = pd.to_datetime(df['month'], format='%m/%Y')
#df.sort_values('month', inplace=True)

In [ ]:
df['year'] = pd.DatetimeIndex(df['month']).year

In [ ]:
df['mon'] = pd.DatetimeIndex(df['month']).month

In [ ]:
df.head()

## новый столбец "mapp"

In [ ]:
# замена незаполненных стран на "A"
df.loc[df['country'].isna(), 'country'] = 'A'
#создание уникального столбца 'mapp'
df['mapp'] = df['region'].astype(str)+ \
             df['district'].astype(str)+ \
             df['country'].astype(str)+ \
             df['code'].astype(str)

# АНАЛИЗ ИСПОЛЬЗОВАННОЙ ПАМЯТИ

In [ ]:
df.info(memory_usage='deep')

In [ ]:
for dtype in ['float','int','object', 'datetime']:
    selected_dtype = df.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

In [ ]:
int_types = ["uint8", "int8", "int16"]
for it in int_types:
    print(np.iinfo(it))

In [ ]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # исходим из предположения о том, что если это не DataFrame, то это Series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # преобразуем байты в мегабайты
    return "{:03.2f} MB".format(usage_mb)

df_int = df.select_dtypes(include=['int64'])
converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(df_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([df_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

In [ ]:
df_float = df.select_dtypes(include=['float64'])
converted_float = df_float.apply(pd.to_numeric,downcast='float')

print(mem_usage(df_float))
print(mem_usage(converted_float))

compare_floats = pd.concat([df_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

In [ ]:
optimized_df = df.copy()

optimized_df[converted_int.columns] = converted_int
optimized_df[converted_float.columns] = converted_float

print(mem_usage(df))
print(mem_usage(optimized_df))

In [ ]:
df_obj = df.select_dtypes(include=['object']).copy()
df_obj.describe()

In [ ]:
converted_obj = pd.DataFrame()

for col in df_obj.columns:
    num_unique_values = len(df_obj[col].unique())
    num_total_values = len(df_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = df_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = df_obj[col]

In [ ]:
print(mem_usage(df_obj))
print(mem_usage(converted_obj))

compare_obj = pd.concat([df_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

In [ ]:
optimized_df[converted_obj.columns] = converted_obj

mem_usage(optimized_df)

In [ ]:
dtypes = optimized_df.drop('month',axis=1).dtypes

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))


preview = first2pairs = {key:value for key,value in list(column_types.items())[:12]}
import pprint
pp = pp = pprint.PrettyPrinter(indent=4)
pp.pprint(preview)

In [ ]:
df.to_csv('data_new.csv', index=False)

In [ ]:
#read_and_optimized = pd.read_csv('data_new.csv',dtype=column_types,parse_dates=['month'],infer_datetime_format=True)

#print(mem_usage(read_and_optimized))
#read_and_optimized.head()

In [ ]:
df.head()